* instalar miniconda
* seguir guia de tensorflow 2.10
* pip install rasterio geopandas scikit-learn albumentations segmentation-models matplotlib tqdm python-dotenv
* pip install "numpy<2.0"

### Creación de gpkg de division

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from utils_pre_procesamiento import generar_tiles_spatial_split

tif_sierra = os.getenv('TIF_SIERRA')
out_dir_sierra = r"D:\...CapasSierra" #ruta completa donde guardar el archivo

tif_selva = os.getenv('TIF_SELVA')
out_dir_selva = r"D:\...CapasSelva"  #ruta completa donde guardar el archivo

generar_tiles_spatial_split(tif_sierra, out_dir = out_dir_sierra)
generar_tiles_spatial_split(tif_selva, out_dir = out_dir_selva)

Píxeles válidos totales: 588882000
Val   asignado: 94217200 (16.0%)
Test asignado: 70662900 (12.0%)
Train: 424001900
Píxeles válidos totales: 591685632
Val   asignado: 118332209 (20.0%)
Test asignado: 70996459 (12.0%)
Train: 402356964


### Creacion de parches

In [ ]:
from raster_to_tile import data_split

tif_selva = os.getenv('TIF_SELVA')
tif_sierra = os.getenv('TIF_SIERRA')

mask_casas_selva_path = os.getenv('GPKG_CASAS_SELVA')
mask_casas_sierra_path = os.getenv('GPKG_CASAS_SIERRA')

mask_caminos_selva_path = os.getenv('GPKG_CAMINOS_SELVA')
mask_caminos_sierra_path = os.getenv('GPKG_CAMINOS_SIERRA')

gpkg_division_selva_tile = os.getenv('GPKG_DIVISION_SELVA')
gpkg_division_sierra_tile = os.getenv('GPKG_DIVISION_SIERRA')

dataset_general_out = r"D:\...Salida"#ruta completa donde guardar los tiles

### Area Excluida
gpkg_area_ciudad = os.getenv('GPKG_AREA_EXCLUIDA')

#Sierra
data_split(tif_sierra, mask_casas_sierra_path, mask_caminos_sierra_path, gpkg_division_sierra_tile, dataset_general_out, gpkg_area_excluida_path=gpkg_area_ciudad)
#Selva
data_split(tif_selva, mask_casas_selva_path, mask_caminos_selva_path, gpkg_division_selva_tile, dataset_general_out)

### Verificar GPU disponible

In [1]:
import tensorflow as tf
print("TF version:", tf.__version__)
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("GPUs:", tf.config.list_physical_devices('GPU'))

TF version: 2.10.1
Built with CUDA: True
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
from utils2 import split_dataset

(train_imgs, train_masks, val_imgs, val_masks, test_imgs, test_masks) = split_dataset(carpeta=dataset_general_out)


División correcta sin errores de archivos.
--------------------------------------------------
TRAIN     → imágenes: 4108 | máscaras: 4108
VALIDATION → imágenes: 1127 | máscaras: 1127
TEST      → imágenes: 812 | máscaras: 812
--------------------------------------------------
TOTAL IMGS: 6047
TOTAL MASKS: 6047
--------------------------------------------------


In [4]:
from TiffDataGeneratorAugmentedUnet import TIFFDataGeneratorAug

# Generadores
train_gen = TIFFDataGeneratorAug(train_imgs, train_masks, normalize='imagenet',batch_size=2, n_channels=3, augment=True)#<--- canales
val_gen = TIFFDataGeneratorAug(val_imgs, val_masks, normalize='imagenet', batch_size=2, shuffle=False, n_channels=3, augment=False)#<--- canales
test_gen = TIFFDataGeneratorAug(test_imgs, test_masks, normalize='imagenet', batch_size=1, shuffle=False, n_channels=3, augment=False)#<--- canales


In [5]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

from tensorflow import keras
import segmentation_models as sm

Segmentation Models: using `tf.keras` framework.


In [ ]:
from tensorflow.keras.optimizers import Adam
import configuraciones as cfg
import tensorflow as tf


BACKBONE = 'resnet34'#efficientnetb0 tmb puede ser (buena con imag satelitales)
model = sm.Unet(
    backbone_name=BACKBONE,
    encoder_weights='imagenet',
    classes=3,
    activation='softmax'
    #input_shape=(512, 512, 4)
)

# Definir nombres de clases
class_names = ['background', 'road', 'building']

# Crear métricas ANTES (para tener referencia)
acc_metric = cfg.MaskedSparseCategoricalAccuracy(
    num_classes=3, 
    class_names=class_names, 
    name='acc'
)

iou_metric = cfg.MeanIoUPerClass(
    num_classes=3, 
    class_names=class_names, 
    name='miou'
)

model.compile(
    optimizer=Adam(1e-4),
    loss=cfg.masked_sparse_cce,
    metrics=[
        acc_metric,
        iou_metric
    ]
)


85521592/85521592 [==============================] - 18s 0us/step


In [7]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

early_stop = EarlyStopping(
    monitor='val_loss',      # métrica que se evalúa
    mode='min',
    patience=10,
    restore_best_weights=True,
    verbose= 1
)

checkpoint = ModelCheckpoint(
    r"checkpoint.keras",         # ruta del archivo
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau( # reduccion lr auto
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

metrics_callback = cfg.PrintMetricsPerClass([acc_metric,iou_metric])


In [9]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=100,
    verbose=1,
    callbacks=[early_stop, checkpoint, reduce_lr, cfg.PrintMetricsPerClass()]
)
print("Epochs realizadas:", len(history.history['loss']))

Epoch 1/100
2054/2054 [==============================] - ETA: 0s - loss: 0.1606 - acc: 0.9633 - miou: 0.4431
Epoch 1: val_loss improved from inf to 0.03745, saving model to checkpoint.keras

Época 1 - Métricas por clase

  📊 Accuracy por clase:
    background: 0.9979 (99.79%)
    road: 0.5973 (59.73%)
    building: 0.5894 (58.94%)

  📊 IoU por clase:
    background: 0.9887 (98.87%)
    road: 0.5341 (53.41%)
    building: 0.5480 (54.80%)

2054/2054 [==============================] - 382s 173ms/step - loss: 0.1606 - acc: 0.9633 - miou: 0.4431 - val_loss: 0.0375 - val_acc: 0.9887 - val_miou: 0.6903 - lr: 1.0000e-04
Epoch 2/100
2054/2054 [==============================] - ETA: 0s - loss: 0.0779 - acc: 0.9808 - miou: 0.6032
Epoch 2: val_loss improved from 0.03745 to 0.03065, saving model to checkpoint.keras

Época 2 - Métricas por clase

  📊 Accuracy por clase:
    background: 0.9969 (99.69%)
    road: 0.6660 (66.60%)
    building: 0.7451 (74.51%)

  📊 IoU por clase:
    background: 0.9903 

In [10]:
model.save("unet_resnet34__model.keras")

In [11]:
model.save_weights("unet_resnet34__model.weights.h5")

In [12]:
# Después del entrenamiento, obtener IoU por clase
iou_metric = model.metrics[-1]  # Última métrica
iou_per_class = iou_metric.get_iou_per_class()

print("\nIoU por clase:")
for class_name, iou_value in iou_per_class.items():
    print(f"  {class_name}: {iou_value:.4f}")


IoU por clase:
  background: 0.9924
  road: 0.7146
  building: 0.6800


In [13]:
# Save final model
model.save("model_final.keras")

# Evaluate on train set
print("\n=== Evaluación en train set ===")
train_results = model.evaluate(train_gen, verbose=1)
print(f"Train Loss: {train_results[0]:.4f}")
print(f"Train Masked Accuracy: {train_results[1]:.4f}")
print(f"Train Masked Mean IoU: {train_results[2]:.4f}")
# Evaluate on test set
print("\n=== Evaluación en val set ===")
val_results = model.evaluate(val_gen, verbose=1)
print(f"Val Loss: {val_results[0]:.4f}")
print(f"Val Masked Accuracy: {val_results[1]:.4f}")
print(f"Val Masked Mean IoU: {val_results[2]:.4f}")
# Evaluate on test set
print("\n=== Evaluación en test set ===")
test_results = model.evaluate(test_gen, verbose=1)
print(f"Test Loss: {test_results[0]:.4f}")
print(f"Test Masked Accuracy: {test_results[1]:.4f}")
print(f"Test Masked Mean IoU: {test_results[2]:.4f}")


=== Evaluación en test set ===
2054/2054 [==============================] - 187s 91ms/step - loss: 0.0620 - acc: 0.9846 - miou: 0.6675
Train Loss: 0.0620
Train Masked Accuracy: 0.9846
Train Masked Mean IoU: 0.6675

=== Evaluación en val set ===
564/564 [==============================] - 37s 65ms/step - loss: 0.0202 - acc: 0.9925 - miou: 0.7953
Val Loss: 0.0202
Val Masked Accuracy: 0.9925
Val Masked Mean IoU: 0.7953

=== Evaluación en test set ===
812/812 [==============================] - 29s 36ms/step - loss: 0.0226 - acc: 0.9913 - miou: 0.7724
Test Loss: 0.0226
Test Masked Accuracy: 0.9913
Test Masked Mean IoU: 0.7724
